# Import Modules

In [1]:
from pymatgen.util.provenance import StructureNL

from pymatgen.io.ase import AseAtomsAdaptor

# Read DFT Data

In [2]:
# #########################################################
import pickle; import os
path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/processing_bulk_dft/creating_final_dataset_for_upload",
    "out_data/df_dft_final_no_dupl.pickle")
with open(path_i, "rb") as fle:
    df_dft_final_no_dupl = pickle.load(fle)
# #########################################################

In [3]:
for id_i, row_i in df_dft_final_no_dupl.iterrows():
    tmp = 42

In [4]:
atoms = row_i.atoms

struct = AseAtomsAdaptor().get_structure(atoms)

In [5]:
authors = [
    {"name": "Raul A. Flores", "email": "raulf2012@gmail.com"},
    {"name": "Chris Paolucci", "email": "cp9wx@virginia.edu"},
    {"name": "Kirsten Winther", "email": "winther@stanford.edu"},
    {"name": "Ankit Jain", "email": "ankitjain.me.iitk@gmail.com"},
    {"name": "Jose Antonio Garrido Torres", "email": "jagt@stanford.edu"},
    {"name": "Muratahan Aykol", "email": "muratahan.aykol@tri.global"},
    {"name": "Joseph Montoya", "email": "joseph.montoya@tri.global"},
    {"name": "Jens Kehlet Nørskov", "email": "jkno@dtu.dk"},
    {"name": "Michal Bajdich", "email": "bajdich@slac.stanford.edu"},
    {"name": "Thomas Bligaard", "email": "tbli@dtu.dk"},
    ]

In [6]:
struct_NL = StructureNL(
    struct,
    authors,
    projects=None,
    references="",
    # remarks=[""],
    data=None,
    history=None,
    created_at="20200329",
    )